In [455]:
import pandas as pd
import json
import numpy as np
import re
from datetime import datetime

In [456]:
with open('Emergency_Response_Raw.json') as data_file:
    raw_data=json.load(data_file)
raw=pd.DataFrame(raw_data['data'])

#raw[0]: index; 1:1812
#raw[7]: '{\n  "invalidCells" : {\n    "28078803" : "40.5863143738225",\n    "28078804" : "-74.1919940234386",
  #\n    "18452410" : "3/30/2015  6:31:05 PM",\n    "18452411" : "3/30/2015   8:09:01PM "\n  }\n}'
#raw[8]: incident type
#raw[9]: location 
#raw[10]: borough

In [478]:
raw7=raw[7]
#raw7.get_values()[i] is not a well defined dictionary object; parse and collect in a list
raw7_list=[json.loads(str(raw7.get_values()[i])) for i in range(len(raw7))]
#line 1812 is not an incident; therefore the "-1"
creation_time_raw=[raw7_list[i].get('invalidCells').get('18452410') for i in range(len(raw7_list)-1)]
closed_time_raw=[raw7_list[i].get('invalidCells').get('18452411') for i in range(len(raw7_list)-1)]

incident_type_raw=list(raw[8][:-1])

invalid_index=[i for i,x in enumerate(closed_time_raw) if x=="Active"]
# -i: due to the shift after pop
for i in range(len(invalid_index)):
    incident_type_raw.pop(invalid_index[i]-i)
    creation_time_raw.pop(invalid_index[i]-i)
    closed_time_raw.pop(invalid_index[i]-i)

1809
1809
1809


In [479]:
incident_type=[incident_type_raw[i].replace(' ','') for i in range(len(incident_type_raw))]
# 275 distinct incident types(detailed)
general_incident_type=[incident_type[i].split('-')[0] for i in range(len(incident_type_raw))]
# 13 distinct general types

# create columns for responsible agencies: FDNY,NYPD,DOHMH,OEM,DOT,DSNY,DEP,DOITT
# number indicates the "level of responsibility", i.e. 1 means the agency takes primary responsibility, 2 secondary...
# the number is smaller than 5
# missing data: assume type "administration"'s responsible agency is OEM
# problem: also missing for categories: fire, lawEnforcement,Marine,Medical,Rescue,Structural...
# agencies=[]
# for i in range(general_incident_type):
#    if general_incident_type[i]=='Administration':
#        agencies[i]=[0,0,0,1,0,0,0,0]
#    if general_incident_type[i]=='Aviation':
#        agencies[i]=[1,2,0,0,0,0,0,0]
#    if general_incident_type[i]=='Fire':
#        agencies[i]=[1,2,0,0,0,0,0,0]



In [480]:
creation_time=[datetime.strptime(creation_time_raw[i], "%m/%d/%Y %I:%M:%S %p") for i in range(len(creation_time_raw))]
closed_time=[datetime.strptime(closed_time_raw[i],"%m/%d/%Y  %I:%M:%S%p ") for i in range(len(closed_time_raw))]

In [481]:
duration_timedelta=[closed_time[i]-creation_time[i] for i in range(len(closed_time))]

In [482]:
duration_hour=[x.total_seconds()/60/60 for x in duration_timedelta]

In [483]:
weekday_or_weekend=[""]*len(creation_time)
season=[""]*len(creation_time)
for i in range(len(creation_time)):
    if creation_time[i].weekday()<5:
        weekday_or_weekend[i]="weekday"
    else:
        weekday_or_weekend[i]="weekend"
        
    if creation_time[i].month<=2 | creation_time[i].month==12:
        season[i]="winter"
    elif creation_time[i].month<=5:
        season[i]="spring"
    elif creation_time[i].month<=8:
        season[i]="summer"
    else:
        season[i]="autumn"

In [484]:
parsed=pd.DataFrame()
parsed['Incident Type']=incident_type
parsed['General Incident Type']=general_incident_type
parsed['Creation Time']=creation_time
parsed['Closed Time']=closed_time
parsed['Duration']=duration_hour
parsed['Weekday or Weekend']=weekday_or_weekend
parsed['Season']=season

In [485]:
parsed

,Incident Type,General Incident Type,Creation Time,Closed Time,Duration,Weekday or Weekend,Season
0,Fire-Brush,Fire,2015-03-30 18:31:05,2015-03-30 20:09:01,1.632222,weekday,spring
1,LawEnforcement-SuspiciousPackage,LawEnforcement,2015-03-30 16:46:33,2015-03-30 19:56:29,3.165556,weekday,spring
2,Fire-1stAlarm,Fire,2015-03-30 04:33:51,2015-03-30 08:02:45,3.481667,weekday,spring
3,Structural-Other,Structural,2015-03-29 17:56:50,2015-03-30 11:56:33,17.995278,weekend,spring
4,Fire-2ndAlarm,Fire,2015-03-29 09:38:45,2015-03-29 10:58:20,1.326389,weekend,spring
5,Utility-Other,Utility,2015-03-28 16:41:44,2015-03-28 18:26:45,1.750278,weekend,spring
6,Utility-GasHighPressure,Utility,2015-03-28 09:48:58,2015-03-29 23:19:03,37.501389,weekend,spring
7,Fire-1stAlarm,Fire,2015-03-28 09:39:03,2015-03-28 12:33:00,2.899167,weekend,spring
8,Fire-2ndAlarm,Fire,2015-03-27 23:36:45,2015-04-06 09:34:40,225.965278,weekday,spring
9,Fire-3rdAlarm,Fire,2015-03-27 01:39:32,2015-03-27 22:13:39,20.568611,weekday,spring
